In [1]:
!python ../../scripts/iuxray/extract_findings_and_impressions.py

Loading file /mnt/workspace/iu-x-ray/dataset/reports/reports.min.json ...
Extracing and merging findings and impressions ...
Saving to /home/pamessina/medvqa-workspace/cache/iuxray/findings+impression.txt ...
done!


In [2]:
!python ../../scripts/mimiccxr/extract_findings_and_impressions.py

Loading MIMIC-CXR's report file paths ...
227835it [02:53, 1309.62it/s] 
Saving findings + impressions to /home/pamessina/medvqa-workspace/cache/mimiccxr/findings+impression.txt ...
100%|██████████████████████████████████| 227835/227835 [07:25<00:00, 510.92it/s]
Num failed paths = 77
Done!


In [3]:
from importlib import reload

In [11]:
from medvqa.utils.files import read_lines_from_txt
from medvqa.utils.common import CACHE_DIR
from medvqa.datasets.qa_pairs_extractor import QuestionAnswerExtractor
import medvqa
import re
import os
import random
import math
from tqdm import tqdm

In [5]:
reload(medvqa.datasets.qa_pairs_extractor)

<module 'medvqa.datasets.qa_pairs_extractor' from '/home/pamessina/medvqa/medvqa/datasets/qa_pairs_extractor.py'>

In [6]:
iuxray_raw_reports = read_lines_from_txt(os.path.join(CACHE_DIR, 'iuxray', 'findings+impression.txt'))

In [7]:
mimiccxr_raw_reports = read_lines_from_txt(os.path.join(CACHE_DIR, 'mimiccxr', 'findings+impression.txt'))

In [8]:
len(iuxray_raw_reports), len(mimiccxr_raw_reports)

(3927, 227758)

In [9]:
def print_qa_pairs(qa_pairs):
    for k, v in qa_pairs.items():
        print('------------')
        print('Q:', k)
        for s in v:
            print('A:',s)

In [10]:
qa_extractor = QuestionAnswerExtractor()

In [11]:
connectors = ['when', 'and', 'but', 'however', 'although', 'should', 'since', 'unless', 'nevertheless']
exp = '('+'|'.join(f'\\b{x}\\b' for x in connectors)+'|[,;.:])'
full_exp = f'{exp}+(\\s+{exp}+)*'
sep_regex = re.compile(full_exp, re.IGNORECASE)
sep_regex

re.compile(r'(\bwhen\b|\band\b|\bbut\b|\bhowever\b|\balthough\b|\bshould\b|\bsince\b|\bunless\b|\bnevertheless\b|[,;.:])+(\s+(\bwhen\b|\band\b|\bbut\b|\bhowever\b|\balthough\b|\bshould\b|\bsince\b|\bunless\b|\bnevertheless\b|[,;.:])+)*',
re.IGNORECASE|re.UNICODE)

In [12]:
def clean_sentence(self, sentence):
    last_sep = None
    i = 0
    n = len(sentence)
    sentence = self._apply_replacements(sentence)
    clean_sentence = ''
    prepend_to_next_chunk = None
    for match in sep_regex.finditer(sentence):
        span = match.span()
        if i < span[0]:
            chunk = sentence[i:span[0]].strip()
            if prepend_to_next_chunk:
                if len(clean_sentence) > 0:
                    chunk = prepend_to_next_chunk + chunk
                prepend_to_next_chunk = None
            if self._is_relevant(chunk):
                if len(clean_sentence) > 0:
                    if last_sep:
                        clean_sentence += ' ' + last_sep
                    clean_sentence += ' '                    
                clean_sentence += chunk
            print('chunk =', chunk)
        i = span[1]
        last_sep = sentence[span[0]:span[1]]
        print('last_sep =', last_sep)
        if last_sep not in ('and', 'but', ':'):
            if last_sep == ', however,':
                last_sep = None
            else:
                done = False
                for x in ('should', 'since', 'unless', 'when'):
                    if x in last_sep:
                        prepend_to_next_chunk = last_sep + ' '
                        last_sep = None
                        done = True
                        break
                if not done:
                    last_sep = ','
    if i < n:
        chunk = sentence[i:].strip()
        if prepend_to_next_chunk:
            chunk = prepend_to_next_chunk + chunk
        if self._is_relevant(chunk):
            if len(clean_sentence) > 0:
                if last_sep:
                    clean_sentence += ' ' + last_sep
                clean_sentence += ' '
            clean_sentence += chunk
        print('chunk =', chunk)
    return clean_sentence

In [13]:
# _s = 'AP chest compared to through : Pulmonary vascular congestion and mediastinal venous engorgement have progressed since .'
_s = 'pa and lateral views of the chest provided demonstrate clear lungs without focal consolidation , effusion or pneumothorax . the cardiomediastinal silhouette is normal . bony structures are intact . no free air below the right hemidiaphragm .'
for x in sep_regex.finditer(_s):
    print(str(x))
print("$$$$"+clean_sentence(qa_extractor, _s)+"$$$$")
print("$$$$"+qa_extractor.clean_sentence(_s)+"$$$$")

<_sre.SRE_Match object; span=(3, 6), match='and'>
<_sre.SRE_Match object; span=(95, 96), match=','>
<_sre.SRE_Match object; span=(122, 123), match='.'>
<_sre.SRE_Match object; span=(167, 168), match='.'>
<_sre.SRE_Match object; span=(196, 197), match='.'>
<_sre.SRE_Match object; span=(240, 241), match='.'>
chunk = pa
last_sep = and
chunk = lateral views of the chest provided demonstrate clear lungs without focal consolidation
last_sep = ,
chunk = effusion or pneumothorax
last_sep = .
chunk = the cardiomediastinal silhouette is normal
last_sep = .
chunk = bony structures are intact
last_sep = .
chunk = no free air below the right hemidiaphragm
last_sep = .
$$$$lateral views of the chest provided demonstrate clear lungs without focal consolidation , effusion or pneumothorax , the cardiomediastinal silhouette is normal , bony structures are intact , no free air below the right hemidiaphragm$$$$
$$$$lateral views of the chest provided demonstrate clear lungs without focal consolidation , e

In [14]:
from nltk.tokenize import sent_tokenize

In [15]:
sent_tokenize('Mild-to-moderate pulmonary edema is new. Right lower lobe opacity has improved consistent with improving atelectasis. Small bilateral pleural effusions are unchanged. Opacity in the left apex and irregularity of the left upper ribs corresponds to mass seen in prior CTA. Cardiomegaly is stable. Increasing opacity in the left lower lobe is consistent with increasing atelectasis and pleural effusion. Spinal hardware is again noted.')

['Mild-to-moderate pulmonary edema is new.',
 'Right lower lobe opacity has improved consistent with improving atelectasis.',
 'Small bilateral pleural effusions are unchanged.',
 'Opacity in the left apex and irregularity of the left upper ribs corresponds to mass seen in prior CTA.',
 'Cardiomegaly is stable.',
 'Increasing opacity in the left lower lobe is consistent with increasing atelectasis and pleural effusion.',
 'Spinal hardware is again noted.']

In [16]:
tmp = qa_extractor.generate_qa_pairs('Previous moderate left pleural effusion and left lower lobe collapse have improved has the patient had thoracentesis?  There is no pneumothorax. Moderate enlarged of the cardiac silhouette is clear due to cardiomegaly and/or pericardial effusion and there is new mild interstitial edema in the right lower lung. Hemodialysis catheter ends in the right atrium as before.', debug=True)
print_qa_pairs(tmp)

------------
Q: pleural space?
A: Previous moderate left pleural effusion
A: Moderate enlarged of the cardiac silhouette is clear due to cardiomegaly and /or pericardial effusion and there is new mild interstitial edema in the right lower lung
------------
Q: pneumothorax?
A: There is no pneumothorax
------------
Q: lungs?
A: Moderate enlarged of the cardiac silhouette is clear due to cardiomegaly and /or pericardial effusion and there is new mild interstitial edema in the right lower lung
------------
Q: heart?
A: Moderate enlarged of the cardiac silhouette is clear due to cardiomegaly and /or pericardial effusion and there is new mild interstitial edema in the right lower lung
A: Hemodialysis catheter ends in the right atrium as before
------------
Q: cardiomegaly?
A: Moderate enlarged of the cardiac silhouette is clear due to cardiomegaly and /or pericardial effusion and there is new mild interstitial edema in the right lower lung
------------
Q: cardiac silhouette?
A: Moderate enla

In [18]:
report = random.choice(mimiccxr_raw_reports)
print(report)
print('=============================')
tmp = qa_extractor.generate_qa_pairs(report, debug=True)
print_qa_pairs(tmp)

The lungs are well expanded and appear clear. There is no focal consolidation, pleural effusion, or pneumothorax. The cardiomediastinal silhouette and hilar contours are unchanged. No acute cardiopulmonary process.
------------
Q: lungs?
A: The lungs are well expanded and appear clear
A: No acute cardiopulmonary process
------------
Q: pleural space?
A: There is no focal consolidation , pleural effusion , or pneumothorax
------------
Q: pneumothorax?
A: There is no focal consolidation , pleural effusion , or pneumothorax
------------
Q: consolidation?
A: There is no focal consolidation , pleural effusion , or pneumothorax
------------
Q: pulmonary hila?
A: The cardiomediastinal silhouette and hilar contours are unchanged
------------
Q: cardiac silhouette?
A: The cardiomediastinal silhouette and hilar contours are unchanged
------------
Q: mediastinum?
A: The cardiomediastinal silhouette and hilar contours are unchanged
------------
Q: heart?
A: No acute cardiopulmonary process


In [19]:
tmp = qa_extractor.generate_qa_pairs_compact_version(report)
tmp

{'sentences': ['The lungs are well expanded and appear clear',
  'There is no focal consolidation , pleural effusion , or pneumothorax',
  'The cardiomediastinal silhouette and hilar contours are unchanged',
  'No acute cardiopulmonary process'],
 'invalid': [],
 'unmatched': [],
 'matched': [0, 1, 2, 3],
 'qa': {'59': [0, 3],
  '68': [1],
  '73': [1],
  '32': [1],
  '75': [2],
  '26': [2],
  '63': [2],
  '48': [3]},
 'question_ids': [59, 68, 73, 32, 75, 26, 63, 48]}

In [21]:
_s = 'The tip of the endotracheal tube projects approximately 3.5 cm above the carina and is within normal limits.'
print("$$$$"+qa_extractor.clean_sentence(_s)+"$$$$")

$$$$The tip of the endotracheal tube projects approximately NUMBER cm above the carina and is normal$$$$


In [22]:
len(qa_extractor.questions), qa_extractor.questions

(97,
 ['ARDS?',
  'COPD?',
  'abscess and cavitation?',
  'adenopathy?',
  'air collections?',
  'air space disease?',
  'air-fluid level?',
  'airways?',
  'apical zone?',
  'ascites?',
  'aspiration?',
  'atelectasis?',
  'azygos lobe?',
  'azygos vein?',
  'bleeding?',
  'blurring?',
  'bones?',
  'bowel obstruction and loops?',
  'bowel?',
  'bronchiectasis?',
  'bronchogram?',
  'bronchovascular crowding?',
  'bronchus?',
  'bullae and blebs?',
  'calcification?',
  'cancer or tumor?',
  'cardiac silhouette?',
  'cardiomegaly?',
  'carina?',
  'cholelithiasis?',
  'colon?',
  'congestive heart failure CHF?',
  'consolidation?',
  'contrast?',
  'densities?',
  'diaphragm?',
  'edema?',
  'emphysema?',
  'esophagus?',
  'fibrosis?',
  'fissures?',
  'fluid overload?',
  'fluid?',
  'fractures?',
  'free air?',
  'gas distension?',
  'granumola?',
  'haze?',
  'heart?',
  'hernias?',
  'infection?',
  'infiltrate?',
  'interstitial lung disease?',
  'kerley lines?',
  'lesions?',
  

In [23]:
tmp = qa_extractor.search_by_unmatched(0, mimiccxr_raw_reports, 100)
x = tmp[1]
tmp

(2, 215937)

In [24]:
qa_extractor.generate_qa_pairs_compact_version(mimiccxr_raw_reports[x])

{'sentences': ['lateral views of the chest provided',
  'Patient is slightly rotated to his left',
  'Bilateral lower lobe airspace consolidation is concerning for pneumonia',
  'No large effusion or pneumothorax',
  'Cardiac silhouette appears mildly enlarged',
  'Mediastinal contour is normal',
  'Imaged osseous structures are intact',
  'No free air below the right hemidiaphragm is seen',
  'Bilateral lower lobe pneumonia'],
 'invalid': [],
 'unmatched': [0, 1],
 'matched': [2, 3, 4, 5, 6, 7, 8],
 'qa': {'59': [2, 8],
  '32': [2],
  '71': [2, 8],
  '68': [3],
  '73': [3],
  '26': [4],
  '63': [5],
  '16': [6],
  '35': [7],
  '44': [7]},
 'question_ids': [59, 32, 71, 68, 73, 26, 63, 16, 35, 44]}

In [25]:
# _report = ""
# print(_report)
# _qa_pairs = qa_generator.generate_question_answer_pairs(_report)
print(mimiccxr_raw_reports[x])
print("======================")
_qa_pairs = qa_extractor.generate_qa_pairs(mimiccxr_raw_reports[x], debug=True)
# _qa_pairs = qa_generator.remove_almost_duplicate_sentences(_qa_pairs)
print_qa_pairs(_qa_pairs)

PA and lateral views of the chest provided. Patient is slightly rotated to his left. Bilateral lower lobe airspace consolidation is concerning for pneumonia. No large effusion or pneumothorax. Cardiac silhouette appears mildly enlarged. Mediastinal contour is normal. Imaged osseous structures are intact. No free air below the right hemidiaphragm is seen. Bilateral lower lobe pneumonia.
** not captured: lateral views of the chest provided
** not captured: Patient is slightly rotated to his left
------------
Q: lungs?
A: Bilateral lower lobe airspace consolidation is concerning for pneumonia
A: Bilateral lower lobe pneumonia
------------
Q: consolidation?
A: Bilateral lower lobe airspace consolidation is concerning for pneumonia
------------
Q: pneumonia?
A: Bilateral lower lobe airspace consolidation is concerning for pneumonia
A: Bilateral lower lobe pneumonia
------------
Q: pleural space?
A: No large effusion or pneumothorax
------------
Q: pneumothorax?
A: No large effusion or pneum

In [26]:
def get_informative_unmatched_sentences(corpus, qa_extractor, n_samples = 2000):
    
    indices = range(len(corpus))
    if len(corpus) > n_samples:
        indices = random.sample(indices, n_samples)
        
    unmatched_sentences = set()
    word2freq = dict()
    
    for i in tqdm(indices):
        for s in qa_extractor.get_unmatched_sentences(corpus[i]):
            unmatched_sentences.add(s)
            for w in re.split(r",?\s+", s):
                word2freq[w] = word2freq.get(w, 0) + 1
    
    max_freq = max(word2freq.values())    
    unmatched_sentences = list(unmatched_sentences)
    
    def score(s):
        score_max = 0
        score_sum = 0
        cnt = 0
        for w in re.split(r",?\s+", s):
            if w not in qa_extractor.stopwords:
                tmp = max_freq - word2freq.get(w, 0)
                score_max = max(tmp, score_max)
                score_sum += tmp
                cnt += 1
        return score_max * 0.4 + (score_sum / math.sqrt(max(cnt,1))) * 0.6
    
    unmatched_sentences.sort(key=score, reverse=True)
    return unmatched_sentences

In [27]:
unmatched_sentences = get_informative_unmatched_sentences(mimiccxr_raw_reports, qa_extractor, n_samples=1000)

100%|██████████| 1000/1000 [00:03<00:00, 269.39it/s]


In [28]:
len(unmatched_sentences)

274

In [29]:
qa_extractor.reload()

In [30]:
print_qa_pairs(qa_extractor.generate_qa_pairs('. '.join(unmatched_sentences[0:40]), debug=True))

** not captured: An explanation could be that the patient during the latest examination interval was mostly located on the right side explaining gravitational forces
** not captured: Oblique views may be obtained for confirmation now , otherwise we suggest a repeat study in 24 hours if patient does not declare clinically
** not captured: The patient has known cardiac chamber enlargement as well as a patent foramen ovale as reported on recent echo
** not captured: No definite acute intrathoracic process , this single AP radiograph is slightly limited due to body habitus
** not captured: The current position represents malposition , given potential damage to the vocal cords
** not captured: Further evaluation with echocardiogram may be warranted in the right clinical setting if this is not been previously evaluated
** not captured: the frontal view is not marked , the accompanying lateral view was obtained with the patient sitting or standing nearly erect
** not captured: The patient's o

In [31]:
!python ../../scripts/iuxray/generate_qa_pairs_from_reports.py

Loading original reports from /mnt/workspace/iu-x-ray/dataset/reports/reports.min.json
Processing each report ...
100%|██████████████████████████████████████| 3955/3955 [00:26<00:00, 147.71it/s]
Saving qa adapted reports to /home/pamessina/medvqa-workspace/cache/iuxray/qa_adapted_reports__20220904_091601.json
Done!


In [37]:
!python ../../scripts/mimiccxr/generate_qa_pairs_from_reports.py

Processing MIMIC-CXR's original reports ...
163634it [26:52, 105.42it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Generate summary file

In [1]:
from medvqa.utils.files import load_json_file

In [3]:
iuxray_qa_adapted_reports = load_json_file('/mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray/qa_adapted_reports__20220904_091601.json')

In [4]:
mimiccxr_qa_adapted_reports = load_json_file('/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/qa_adapted_reports__20220904_095810.json')

In [5]:
len(iuxray_qa_adapted_reports['questions']), len(iuxray_qa_adapted_reports['reports'])

(97, 3927)

In [6]:
len(mimiccxr_qa_adapted_reports['questions']), len(mimiccxr_qa_adapted_reports['reports'])

(97, 227835)

In [7]:
iuxray_qa_adapted_reports['reports'][0]

{'sentences': ['The lungs are hyperexpanded , with increased AP diameter of the chest',
  'The cardiomediastinal silhouette is stable and normal',
  'There is no pneumothorax or large pleural effusion',
  'No acute cardiopulmonary abnormality',
  'Chronic changes consistent with emphysema'],
 'invalid': [],
 'unmatched': [],
 'matched': [0, 1, 2, 3, 4],
 'qa': {'59': [0, 3],
  '26': [1],
  '63': [1],
  '68': [2],
  '73': [2],
  '48': [3],
  '37': [4]},
 'question_ids': [59, 26, 63, 68, 73, 48, 37],
 'filename': '2509.xml',
 'background': 'XXXX-year-old XXXX with chest pain.'}

In [8]:
mimiccxr_qa_adapted_reports['reports'][-1000]

{'sentences': ['Portable AP semi-erect chest radiograph was obtained',
  'Lung volumes continues to be low',
  'The heart continues to be moderately enlarged with unchanged mild to moderate edema',
  'Moderate bilateral , right greater than left , pleural effusions are unchanged',
  'ET tube is in appropriate position , the gastric tube ends in the stomach',
  'Left central venous line is in appropriate position',
  'Right PICC line ends at the lower SVC',
  'Unchanged low lung volumes , mild pulmonary edema and bilateral moderate pleural effusions'],
 'invalid': [],
 'unmatched': [0],
 'matched': [1, 2, 3, 4, 5, 6, 7],
 'qa': {'59': [1, 7],
  '48': [2],
  '36': [2, 7],
  '68': [3, 7],
  '95': [4, 5, 6],
  '87': [4, 5, 6],
  '85': [4],
  '96': [6]},
 'question_ids': [59, 48, 36, 68, 95, 87, 85, 96],
 'filepath': '/mnt/data/mimic-cxr/files/p19/p19830694/s59684066.txt',
 'background': 'history: -year-old man with ET tube, fluid overload. Evaluate for interval change. Portable AP.'}

In [9]:
def generate_summary_file(dataset, k, kk, output_path):
    
    q2a = [[] for _ in range(len(dataset['questions']))]
    n_invalid = 0
    unmatched_sentences = []
    for ri, report in enumerate(dataset['reports']):
        s = report['sentences']
        for q_idx, a_idxs in report['qa'].items():
            q_idx = int(q_idx)
            a_len = sum(len(s[i]) for i in a_idxs)
            q2a[q_idx].append((a_len, ri))
        n_invalid += len(report['invalid'])
        unmatched_sentences.extend(s[i] for i in report['unmatched'])
            
    sorted_questions = sorted([(len(a), i) for i,a in enumerate(q2a)], reverse=True)    
    
    with open(output_path, 'w') as file:
        file.write('Preguntas ordenadas por frecuencia (decreciente)\n\n')
        for f, q in sorted_questions:
            file.write('%d, %s\n' % (f, dataset['questions'][q]))
        file.write('\n')
        file.write('Ejemplos de respuestas por pregunta:')
        for i,(_,q) in enumerate(sorted_questions):
            answers = q2a[q]
            answers.sort()            
            tmp = []
            for _, a in answers:
                report = dataset['reports'][a]
                sentences = report['sentences']
                a_idxs = report['qa'][str(q)]
                answer = '. '.join(sentences[i] for i in a_idxs)
                tmp.append(answer)
            answers = tmp            
            n = len(answers)
            print("n=%d, %s" % (n, dataset['questions'][q]))
            if n > k:
                sample = []
                for j in range(k):
                    sample.append(answers[random.randint(int(n*j/k), int(n*(j+1)/k)-1)])
            else:
                sample = answers
            file.write('\n\n%d) %s:\n\n' % (i+1, dataset['questions'][q]))
            for a in sample:
                file.write('    A: %s\n' % a)
        file.write('\n\nTotal oraciones inválidas: %d\n' % n_invalid)
        file.write('Total oraciones sin match: %d\n' % len(unmatched_sentences))
        file.write('\n\nEjemplos de oraciones válidas que no hicieron match con ninguna pregunta\n\n')
        unmatched_sentences = list(set(unmatched_sentences))
        unmatched_sentences.sort(key=lambda s:len(s))
        nn = len(unmatched_sentences)
        for j in range(kk):
            file.write('    %s\n' % unmatched_sentences[random.randint(int(nn*j/kk), int(nn*(j+1)/kk)-1)])

In [12]:
generate_summary_file(iuxray_qa_adapted_reports, 20, 120, '/home/pamessina/medvqa/medvqa/datasets/iuxray/iuxray_qa_dataset_summary_file.txt')

n=3551, lungs?
n=3371, heart?
n=3106, pleural space?
n=2691, pneumothorax?
n=2360, mediastinum?
n=1967, bones?
n=1243, consolidation?
n=1185, cardiac silhouette?
n=815, pulmonary vascularity?
n=757, opacities?
n=601, calcification?
n=552, air space disease?
n=457, granumola?
n=440, support devices and foreign bodies?
n=431, infiltrate?
n=397, thoracic aorta?
n=395, atelectasis?
n=354, edema?
n=341, nodules?
n=305, cardiomegaly?
n=273, surgery?
n=268, pneumonia?
n=249, pulmonary hila?
n=244, diaphragm?
n=235, scarring?
n=226, tortuosity?
n=220, tubes and lines?
n=220, interstitial lung disease?
n=209, fractures?
n=194, ribs?
n=188, masses?
n=155, prominence?
n=149, densities?
n=146, soft tissue?
n=122, emphysema?
n=113, apical zone?
n=111, trachea?
n=103, bronchus?
n=100, adenopathy?
n=89, markings?
n=83, bronchovascular crowding?
n=75, fluid?
n=71, infection?
n=70, air collections?
n=64, free air?
n=63, lesions?
n=56, hernias?
n=55, vena cava?
n=49, lung parenchyma?
n=49, COPD?
n=44, l

In [14]:
generate_summary_file(mimiccxr_qa_adapted_reports, 20, 120, '/home/pamessina/medvqa/medvqa/datasets/mimiccxr/mimiccxr_qa_dataset_summary_file.txt')

n=195450, lungs?
n=187598, pleural space?
n=147038, pneumothorax?
n=131916, heart?
n=121099, mediastinum?
n=109798, support devices and foreign bodies?
n=100728, cardiac silhouette?
n=88316, tubes and lines?
n=75213, bones?
n=72255, opacities?
n=71586, consolidation?
n=70460, atelectasis?
n=66303, edema?
n=55357, pulmonary hila?
n=54769, pulmonary vascularity?
n=54753, pneumonia?
n=32604, cardiomegaly?
n=28386, surgery?
n=26564, diaphragm?
n=22876, thoracic aorta?
n=21104, vena cava?
n=17690, stomach?
n=16618, trachea?
n=16061, interstitial lung disease?
n=14877, calcification?
n=14737, apical zone?
n=14274, fractures?
n=14140, infection?
n=12334, carina?
n=11983, lung parenchyma?
n=11940, ribs?
n=11769, prominence?
n=11488, tortuosity?
n=9707, free air?
n=9551, fluid?
n=9077, bronchus?
n=8660, aspiration?
n=8229, nodules?
n=7984, scarring?
n=7663, densities?
n=5755, emphysema?
n=5713, markings?
n=5694, congestive heart failure CHF?
n=5079, infiltrate?
n=4709, esophagus?
n=4562, masses